In [ ]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
    'width': 1024,
    'height': 900,
    'start_slideshow_at': 'selected',
});

%matplotlib inline
import seaborn as sns
import numpy as np
import threading
from time import sleep
from IPython.display import display, clear_output, HTML
import ipywidgets

sns.set(rc={'figure.figsize': (15, 6)})
np.set_printoptions(precision=5, threshold=100)

# Set up some variables in case we forget to define them later

import epics

temperature_pv = epics.PV('PYEPICS_DEMO:TEMPERATURE')
setpoint_pv = epics.PV('PYEPICS_DEMO:SETPOINT')
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR')
wave_pv = epics.PV('PYEPICS_DEMO:WAVE')


def clear_callbacks():
    for pv in [temperature_pv, colour_pv, wave_pv]:
        pv.clear_callbacks()
        pv.connection_callback = []
        
display(HTML("""
<style>
    .reveal pre {
        line-height: 1.4em !important;
    }
</style>
"""))

# Why PyEPICS?

## Because Python is awesome!

![http://xkcd.com/353/](images/xkcd-800.png)

`http://xkcd.com/353/`

* Easy to learn
* Highly readable syntax
* Powerful features
* Widely used at many research facilities

### PyEPICS enables you to...

* Quickly access data from EPICS PVs in convenient formats (eg numpy arrays)
* Easily create Python representations of physical devices
* Automate interactions with multiple devices
* Avoid the low-level details of the channel access protocol and focus on your application

![PyEPICS](images/pyepics.png)

# Setup

### Dependencies

* [EPICS Base](http://www.aps.anl.gov/epics/base/R3-14/12.php) libraries
* Python 2.6+ or 3.2+... ***2.7 or 3.4+ highly recommended***
* *Optional:* numpy for nicer array handling

### Installation

In your terminal:

```bash
pip3 install pyepics
```

Set the environment variables (eg in your `~/.bashrc` file):

```bash
export EPICS_BASE=/epics/base
export EPICS_HOST_ARCH=linux-x86_64
```

# Let's get some PVs

In [ ]:
import epics

In [ ]:
temperature_pv = epics.PV('PYEPICS_DEMO:TEMPERATURE')

In [ ]:
temperature_pv.get()

In [ ]:
temperature_pv.value

## Other PV properties

In [ ]:
temperature_pv.units

In [ ]:
print('Low alarm limit:', temperature_pv.lower_alarm_limit)
print('High alarm limit:', temperature_pv.upper_alarm_limit)

In [ ]:
print('Alarm severity:', temperature_pv.severity) 

In [ ]:
temperature_pv.timestamp

# Getting Stringy PVs

* Strings
* Enums
* Character arrays

In [ ]:
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR')
colour_pv.get()

In [ ]:
colour_pv.get(as_string=True)

In [ ]:
colour_pv.char_value

***Warning:*** `pv.char_value` has a bug where it only works after doing `pv.get(as_string=True)`

# Arrays and waveforms

In [ ]:
wave_pv = epics.PV('PYEPICS_DEMO:WAVE')
data = wave_pv.get()
data

In [ ]:
print(data.min(), data.max(), data.mean())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(data)

# &nbsp;

# &nbsp;

# &nbsp;

## What happens if a PV doesn't exist

In [ ]:
invalid_pv = epics.PV('INVALID_NAME')
value = invalid_pv.get()
print(value)

In [ ]:
if value is not None:
    print('We got a value :)')
else:
    print('Failed to get a value :(')

### Lesson

```python
if not binary_pv.get():  # DON'T DO THIS
    do_something()

if binary_pv.get() == 0:  # DO THIS
    do_something()
```

# Setting Values

In [ ]:
setpoint_pv = epics.PV('PYEPICS_DEMO:SETPOINT')

In [ ]:
setpoint_pv.put(8.2)

In [ ]:
setpoint_pv.value = 2.3

### Enum PVs accept int or str

In [ ]:
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR')

In [ ]:
colour_pv.put(0)

In [ ]:
colour_pv.put('blue')

## Warning: Put doesn't guarantee the value was set

### When outside of drive limits

In [ ]:
setpoint_pv.put(15)  # PYEPICS_DEMO:SETPOINT has a DRVH of 10

In [ ]:
setpoint_pv.value

### When the PV is disconnected

In [ ]:
disconnected_pv = epics.PV('INVALID_NAME')

In [ ]:
status = disconnected_pv.put(3)
print(status)

# Callback functions

Notifications when PVs state changes:

* Connection
* Value
* Alarm

## Connection callbacks

In [ ]:
def on_connection_change(pvname, conn, pv):
    print(pvname, 'connected:', conn, flush=True)

In [ ]:
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR',
                     connection_callback=on_connection_change)

## Value callbacks

In [ ]:
def on_value(pvname, value, char_value,
             timestamp, **other_params):
    
    print(pvname, timestamp, value, char_value, flush=True)
    print('severity:', other_params['severity'],
          'status:', other_params['status'], flush=True)

#### Add the callback to the PV

In [ ]:
callback_id = colour_pv.add_callback(on_value)
print('Callback was added with id:', callback_id)

#### Remove the callback

In [ ]:
colour_pv.remove_callback(callback_id)

#### Remove all callbacks

In [ ]:
colour_pv.clear_callbacks()

# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;

### What *not* to do inside callback functions

Any other channel access function calls

Slow / resource intensive processing; the callback should complete in ms

## Running a task from a callback:
## The *wrong* way

In [ ]:
temperature_textbox = ipywidgets.Text()
task_textbox = ipywidgets.Text()

display(temperature_textbox, task_textbox)

def temperature_callback(value, **kwargs):
    temperature_textbox.value = str(value)
    
def colour_callback(char_value, **kwargs):
    task_textbox.value = 'RUNNING SLOW TASK'
    sleep(5)
    task_textbox.value = 'DONE!'

temperature_pv.add_callback(temperature_callback)
colour_pv.add_callback(colour_callback);

In [ ]:
temperature_pv.clear_callbacks()
colour_pv.clear_callbacks()

# Options

* Spawn a new thread from the callback.

* Trigger processing on a worker thread using a queue or signal. See:

    * `queue` package:
    https://docs.python.org/3/library/queue.html
    * `threading` package: https://docs.python.org/3/library/threading.html

# `threading.Event`

* Enables signalling between threads
* Calling `event.wait()` will pause execution until the event is triggered
* Calling `event.set()` signals the event has happened, allowing any threads that were waiting for it to run

## Running a task from a callback:
## The right way!

In [ ]:
COLOUR_CHANGE = threading.Event()

def worker():
    while True:
        COLOUR_CHANGE.wait()  # <= Blocks until event is triggered
        task_textbox.value = 'RUNNING SLOW TASK'
        sleep(5)
        task_textbox.value = 'DONE!'
        COLOUR_CHANGE.clear()

worker_thread = epics.ca.CAThread(target=worker)
worker_thread.start()

def temperature_callback(value, **kwargs):
    temperature_textbox.value = str(value)
    
def colour_callback(char_value, **kwargs):
    COLOUR_CHANGE.set()  # <= Trigger the event

display(temperature_textbox, task_textbox)
temperature_pv.add_callback(temperature_callback)
colour_pv.add_callback(colour_callback);

In [ ]:
temperature_pv.clear_callbacks()
colour_pv.clear_callbacks()

## Devices

Structure for reading and writing related PVs

In [ ]:
demo_device = epics.Device(prefix='PYEPICS_DEMO:')
print(demo_device)

In [ ]:
demo_device.TEMPERATURE

In [ ]:
demo_device.COLOUR

In [ ]:
demo_device.get('COLOUR', as_string=True)

In [ ]:
demo_device.SETPOINT = 8

# Under the covers

In [ ]:
print(demo_device)

In [ ]:
print(demo_device._pvs)

# Danger!

In [ ]:
demo_device.INVALID_NAME

# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;

## Defensive programming with the Device class

In [ ]:
safe_demo_device = epics.Device(prefix='PYEPICS_DEMO:',
                                aliases={'target': 'SETPOINT',
                                         'value': 'READBACK',
                                         'colour': 'COLOUR'},
                                mutable=False)
print(safe_demo_device)

In [ ]:
safe_demo_device.target = 1.5

In [ ]:
safe_demo_device.invalid_name

# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;

# Saving device state

In [ ]:
state = safe_demo_device.save_state()
state

In [ ]:
safe_demo_device.SETPOINT = 7.1

In [ ]:
safe_demo_device.restore_state(state)

In [ ]:
safe_demo_device.write_state('demo_device_state.txt')

## Subclassing Device

In [ ]:
class DemoDevice(epics.Device):
    def __init__(self, aliases=None, mutable=False, **kws):
        if aliases is None:
            aliases = {}
        aliases.update({'colour': 'COLOUR',
                        'target': 'SETPOINT',
                        'readback': 'READBACK'})
        super().__init__(aliases=aliases, mutable=mutable, **kws)
    
    def go_to_setpoint(self, setpoint):
        self.target = setpoint
        while abs(self.readback - setpoint) > .1:
            print('Moving...', flush=True)
            sleep(.5)
            
    @property
    def colour_string(self):
        return self.get('colour', as_string=True)

demo_device = DemoDevice(prefix='PYEPICS_DEMO:')

In [ ]:
demo_device.go_to_setpoint(7.5)

In [ ]:
demo_device.colour_string

# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;

## Motor device class

```python
>>> from epics import Motor
>>> motor_x = Motor('SR00ID00USR00:MOT8')
>>> motor_x.moving
0
>>> motor_x.readback
10.5
>>> motor_x.slew_speed = 100
>>> motor_x.move(123, wait=True)
0
```

# Advanced Topics

## Getting many PVs

In [ ]:
names = ['PYEPICS_DEMO:VAL{:02d}'.format(n)
         for n in range(100)]
print(names[:5])

In [ ]:
%%time
epics.ca.clear_cache()

values = []
for name in names:
    values.append(epics.PV(name).get())
print('TOTAL:', sum(values))

In [ ]:
%%time
epics.ca.clear_cache()

values = epics.caget_many(names)
print('TOTAL:', sum(values))

# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;
# &nbsp;

## Handling large arrays

Avoid setting up unnecessary monitors:

```python
>>> name = '03ID:SAMPLE:image1:ArrayData'
>>> data_pv = epics.PV(name,
...                    auto_monitor=False)
>>> image = data_pv.get()
```

# Other features

* Alarm class http://pyepics.github.io/pyepics/alarm.html
* wxPython Widgets http://pyepics.github.io/pyepics/wx.html
* autosave http://pyepics.github.io/pyepics/autosave.html
* PyEPICS Applications http://pyepics.github.io/epicsapps/

# Final thoughts

* Please ask questions and share experiences.
* Documentation is very good:
  https://pyepics.github.io/pyepics/
* The PyEPICS code is very readable dive in at:
  https://github.com/pyepics/pyepics
* Don't be afraid to report bugs!